In [108]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow import keras
import os
import datetime
from gym import wrappers

In [109]:
class MyModel(tf.keras.Model):
    def __init__(self, num_states, hidden_units, num_actions):
        super(MyModel, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(input_shape=(num_states,))
        self.hidden_layers = []
        for i in hidden_units:
            self.hidden_layers.append(Dense(i, activation='tanh', kernel_initializer='RandomNormal'))
        self.output_layer = Dense(num_actions, activation='linear', kernel_initializer='RandomNormal')
        
    @tf.function
    def call(self, inputs):
        z = self.input_layer(inputs)
        for layer in self.hidden_layers:
            z = layer(z)
        output = self.output_layer(z)
        return output

In [110]:
class DQN:
    def __init__(self, num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr):
        self.num_actions = num_actions
        self.batch_size = batch_size
        self.optimizer = tf.optimizers.Adam(lr)
        self.gamma = gamma
        self.model = MyModel(num_states, hidden_units, num_actions)
        self.experience = {'s': [], 'a': [], 'r': [], 's2': [], 'done': []}  # done (bool): is current state terminal
        self.max_experiences = max_experiences
        self.min_experiences = min_experiences
         
    def predict(self, inputs):
        return self.model(np.atleast_2d(inputs.astype('float32')))
    
    def train(self, TargetNet):
        if len(self.experience['s']) < self.min_experiences:
            return 0
        ids = np.random.randint(low=0, high=len(self.experience['s']), size=self.batch_size)
        states = np.asarray([self.experience['s'][i] for i in ids])
        actions = np.asarray([self.experience['a'][i] for i in ids])
        rewards = np.asarray([self.experience['r'][i] for i in ids])
        states_next = np.asarray([self.experience['s2'][i] for i in ids])
        dones = np.asarray([self.experience['done'][i] for i in ids])
        value_next = np.max(TargetNet.predict(states_next), axis=1)
        actual_values = np.where(dones, rewards, rewards + self.gamma*value_next)
        
        with tf.GradientTape() as tape:
            selected_action_values = tf.math.reduce_sum(
                self.predict(states) * tf.one_hot(actions, self.num_actions), axis=1)
            loss = tf.math.reduce_mean(tf.square(actual_values - selected_action_values))
        variables = self.model.trainable_variables
        gradients = tape.gradient(loss, variables)
        self.optimizer.apply_gradients(zip(gradients, variables))
        return loss
        
    def get_action(self, states, epsilon):
        if np.random.random() < epsilon:
            return np.random.choice(self.num_actions)
        else:
            return np.argmax(self.predict(np.atleast_2d(states))[0])


    def add_experience(self, exp):
        if len(self.experience['s']) >= self.max_experiences:
            for key in self.experience.keys():
                self.experience[key].pop(0)
        for key, value in exp.items():
            self.experience[key].append(value)

    def copy_weights(self, TrainNet):
        variables1 = self.model.trainable_variables
        variables2 = TrainNet.model.trainable_variables
        for v1, v2 in zip(variables1, variables2):
            v1.assign(v2.numpy())

In [111]:
def play_game(env, TrainNet, TargetNet, epsilon, copy_step):
    rewards = 0
    iter = 0
    done = False
    observations = env.reset()
    losses = list()
    while not done:
        action = TrainNet.get_action(observations, epsilon)
        prev_observations = observations
        observations, reward, done, _ = env.step(action)
        rewards += reward
        if done:
            reward = -200
            env.reset()

        exp = {'s': prev_observations, 'a': action, 'r': reward, 's2': observations, 'done': done}
        TrainNet.add_experience(exp)
        loss = TrainNet.train(TargetNet)
        if isinstance(loss, int):
            losses.append(loss)
        else:
            losses.append(loss.numpy())
        iter += 1
        if iter % copy_step == 0:
            TargetNet.copy_weights(TrainNet)
    return rewards, np.mean(losses)

In [112]:
def make_video(env, TrainNet):
    env = wrappers.Monitor(env, os.path.join(os.getcwd(), "videos"), force=True)
    rewards = 0
    steps = 0
    done = False
    observation = env.reset()
    while not done:
        action = TrainNet.get_action(observation, 0)
        observation, reward, done, _ = env.step(action)
        steps += 1
        rewards += reward
    print("Testing steps: {} rewards {}: ".format(steps, rewards))

In [113]:
def main():
    env = gym.make('CartPole-v0')
    gamma = 0.99
    copy_step = 25
    num_states = len(env.observation_space.sample())
    num_actions = env.action_space.n
    hidden_units = [200, 200]
    max_experiences = 10000
    min_experiences = 100
    batch_size = 32
    lr = 1e-2
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    log_dir = 'logs/dqn/' + current_time
    summary_writer = tf.summary.create_file_writer(log_dir)

    TrainNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)
    TargetNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)
    N = 50000
    total_rewards = np.empty(N)
    epsilon = 0.99
    decay = 0.9999
    min_epsilon = 0.1
    for n in range(N):
        epsilon = max(min_epsilon, epsilon * decay)
        total_reward, losses = play_game(env, TrainNet, TargetNet, epsilon, copy_step)
        total_rewards[n] = total_reward
        avg_rewards = total_rewards[max(0, n - 100):(n + 1)].mean()
        with summary_writer.as_default():
            tf.summary.scalar('episode reward', total_reward, step=n)
            tf.summary.scalar('running avg reward(100)', avg_rewards, step=n)
            tf.summary.scalar('average loss)', losses, step=n)
        if True:  # n % 100 == 0:
            print("episode:", n, "episode reward:", total_reward, "epsilon:", epsilon, "avg reward (last 100):", avg_rewards,
                  "episode loss: ", losses)
    print("avg reward for last 100 episodes:", avg_rewards)
    make_video(env, TrainNet)
    env.close()

In [ ]:
if __name__ == '__main__':
    main()

episode: 0 episode reward: 21.0 epsilon: 0.989901 avg reward (last 100): 21.0 episode loss:  0.0
episode: 1 episode reward: 22.0 epsilon: 0.9898020099 avg reward (last 100): 21.5 episode loss:  0.0
episode: 2 episode reward: 31.0 epsilon: 0.98970302969901 avg reward (last 100): 24.666666666666668 episode loss:  0.0
episode: 3 episode reward: 22.0 epsilon: 0.9896040593960401 avg reward (last 100): 24.0 episode loss:  0.0
episode: 4 episode reward: 18.0 epsilon: 0.9895050989901005 avg reward (last 100): 22.8 episode loss:  821.255954530504
episode: 5 episode reward: 14.0 epsilon: 0.9894061484802015 avg reward (last 100): 21.333333333333332 episode loss:  1603.0071
episode: 6 episode reward: 11.0 epsilon: 0.9893072078653534 avg reward (last 100): 19.857142857142858 episode loss:  2103.9795
episode: 7 episode reward: 19.0 epsilon: 0.9892082771445669 avg reward (last 100): 19.75 episode loss:  1990.7671
episode: 8 episode reward: 14.0 epsilon: 0.9891093563168525 avg reward (last 100): 19.11

episode: 67 episode reward: 21.0 epsilon: 0.9832905026656819 avg reward (last 100): 22.897058823529413 episode loss:  626.6761
episode: 68 episode reward: 16.0 epsilon: 0.9831921736154153 avg reward (last 100): 22.797101449275363 episode loss:  761.53296
episode: 69 episode reward: 10.0 epsilon: 0.9830938543980539 avg reward (last 100): 22.614285714285714 episode loss:  510.57993
episode: 70 episode reward: 18.0 epsilon: 0.9829955450126141 avg reward (last 100): 22.549295774647888 episode loss:  639.3327
episode: 71 episode reward: 18.0 epsilon: 0.9828972454581129 avg reward (last 100): 22.48611111111111 episode loss:  580.90717
episode: 72 episode reward: 11.0 epsilon: 0.982798955733567 avg reward (last 100): 22.328767123287673 episode loss:  569.0018
episode: 73 episode reward: 14.0 epsilon: 0.9827006758379937 avg reward (last 100): 22.216216216216218 episode loss:  502.8875
episode: 74 episode reward: 17.0 epsilon: 0.9826024057704099 avg reward (last 100): 22.14666666666667 episode 

episode: 132 episode reward: 21.0 epsilon: 0.9769195239571736 avg reward (last 100): 23.693069306930692 episode loss:  290.49994
episode: 133 episode reward: 17.0 epsilon: 0.9768218320047779 avg reward (last 100): 23.742574257425744 episode loss:  392.43628
episode: 134 episode reward: 13.0 epsilon: 0.9767241498215774 avg reward (last 100): 23.702970297029704 episode loss:  263.12408
episode: 135 episode reward: 17.0 epsilon: 0.9766264774065953 avg reward (last 100): 23.475247524752476 episode loss:  242.25456
episode: 136 episode reward: 17.0 epsilon: 0.9765288147588546 avg reward (last 100): 23.504950495049506 episode loss:  302.56952
episode: 137 episode reward: 11.0 epsilon: 0.9764311618773787 avg reward (last 100): 23.366336633663366 episode loss:  193.82422
episode: 138 episode reward: 16.0 epsilon: 0.9763335187611909 avg reward (last 100): 23.207920792079207 episode loss:  374.81012
episode: 139 episode reward: 17.0 epsilon: 0.9762358854093148 avg reward (last 100): 23.217821782

episode: 196 episode reward: 11.0 epsilon: 0.9706868930610726 avg reward (last 100): 23.128712871287128 episode loss:  354.92273
episode: 197 episode reward: 32.0 epsilon: 0.9705898243717664 avg reward (last 100): 23.346534653465348 episode loss:  346.61514
episode: 198 episode reward: 24.0 epsilon: 0.9704927653893293 avg reward (last 100): 23.0 episode loss:  399.4546
episode: 199 episode reward: 12.0 epsilon: 0.9703957161127903 avg reward (last 100): 22.95049504950495 episode loss:  272.55792
episode: 200 episode reward: 11.0 epsilon: 0.9702986765411791 avg reward (last 100): 22.85148514851485 episode loss:  269.54056
episode: 201 episode reward: 21.0 epsilon: 0.9702016466735249 avg reward (last 100): 22.940594059405942 episode loss:  409.76996
episode: 202 episode reward: 18.0 epsilon: 0.9701046265088575 avg reward (last 100): 22.534653465346533 episode loss:  297.17114
episode: 203 episode reward: 16.0 epsilon: 0.9700076160462067 avg reward (last 100): 22.306930693069308 episode lo

episode: 261 episode reward: 14.0 epsilon: 0.9643975762095879 avg reward (last 100): 20.871287128712872 episode loss:  291.67145
episode: 262 episode reward: 40.0 epsilon: 0.9643011364519669 avg reward (last 100): 21.138613861386137 episode loss:  308.75153
episode: 263 episode reward: 14.0 epsilon: 0.9642047063383217 avg reward (last 100): 20.96039603960396 episode loss:  204.1534
episode: 264 episode reward: 17.0 epsilon: 0.9641082858676878 avg reward (last 100): 20.93069306930693 episode loss:  271.23706
episode: 265 episode reward: 40.0 epsilon: 0.964011875039101 avg reward (last 100): 21.14851485148515 episode loss:  223.88058
episode: 266 episode reward: 21.0 epsilon: 0.9639154738515971 avg reward (last 100): 21.198019801980198 episode loss:  215.32977
episode: 267 episode reward: 25.0 epsilon: 0.963819082304212 avg reward (last 100): 21.178217821782177 episode loss:  263.98935
episode: 268 episode reward: 41.0 epsilon: 0.9637227003959816 avg reward (last 100): 21.247524752475247

...

episode: 4327 episode reward: 14.0 epsilon: 0.6421894166507901 avg reward (last 100): 18.504950495049506 episode loss:  181.52087
episode: 4328 episode reward: 19.0 epsilon: 0.642125197709125 avg reward (last 100): 18.554455445544555 episode loss:  222.0552
episode: 4329 episode reward: 9.0 epsilon: 0.6420609851893541 avg reward (last 100): 18.15841584158416 episode loss:  178.27325
episode: 4330 episode reward: 13.0 epsilon: 0.6419967790908351 avg reward (last 100): 18.15841584158416 episode loss:  186.02026
episode: 4331 episode reward: 16.0 epsilon: 0.641932579412926 avg reward (last 100): 18.128712871287128 episode loss:  194.99414
episode: 4332 episode reward: 20.0 epsilon: 0.6418683861549848 avg reward (last 100): 18.217821782178216 episode loss:  252.66824
episode: 4333 episode reward: 33.0 epsilon: 0.6418041993163692 avg reward (last 100): 18.415841584158414 episode loss:  251.95892
episode: 4334 episode reward: 34.0 epsilon: 0.6417400188964376 avg reward (last 100): 18.5643564

episode: 4392 episode reward: 30.0 epsilon: 0.6380285149750188 avg reward (last 100): 18.821782178217823 episode loss:  196.45226
episode: 4393 episode reward: 19.0 epsilon: 0.6379647121235212 avg reward (last 100): 18.81188118811881 episode loss:  229.77673
episode: 4394 episode reward: 15.0 epsilon: 0.6379009156523089 avg reward (last 100): 18.73267326732673 episode loss:  150.35487
episode: 4395 episode reward: 15.0 epsilon: 0.6378371255607437 avg reward (last 100): 18.73267326732673 episode loss:  196.2145
episode: 4396 episode reward: 19.0 epsilon: 0.6377733418481876 avg reward (last 100): 18.762376237623762 episode loss:  201.24525
episode: 4397 episode reward: 27.0 epsilon: 0.6377095645140027 avg reward (last 100): 18.93069306930693 episode loss:  225.44942
episode: 4398 episode reward: 12.0 epsilon: 0.6376457935575514 avg reward (last 100): 18.861386138613863 episode loss:  200.771
episode: 4399 episode reward: 34.0 epsilon: 0.6375820289781956 avg reward (last 100): 18.74257425

episode: 4457 episode reward: 22.0 epsilon: 0.6338945727948517 avg reward (last 100): 20.792079207920793 episode loss:  201.95366
episode: 4458 episode reward: 35.0 epsilon: 0.6338311833375723 avg reward (last 100): 20.96039603960396 episode loss:  261.587
episode: 4459 episode reward: 13.0 epsilon: 0.6337678002192385 avg reward (last 100): 20.762376237623762 episode loss:  215.19019
episode: 4460 episode reward: 33.0 epsilon: 0.6337044234392166 avg reward (last 100): 20.821782178217823 episode loss:  245.83461
episode: 4461 episode reward: 12.0 epsilon: 0.6336410529968727 avg reward (last 100): 20.683168316831683 episode loss:  280.62375
episode: 4462 episode reward: 10.0 epsilon: 0.633577688891573 avg reward (last 100): 20.594059405940595 episode loss:  325.59845
episode: 4463 episode reward: 44.0 epsilon: 0.6335143311226838 avg reward (last 100): 20.77227722772277 episode loss:  298.77563
episode: 4464 episode reward: 10.0 epsilon: 0.6334509796895715 avg reward (last 100): 20.732673

episode: 4522 episode reward: 19.0 epsilon: 0.6297874154331495 avg reward (last 100): 20.02970297029703 episode loss:  234.72081
episode: 4523 episode reward: 13.0 epsilon: 0.6297244366916063 avg reward (last 100): 20.04950495049505 episode loss:  177.76898
episode: 4524 episode reward: 33.0 epsilon: 0.6296614642479371 avg reward (last 100): 20.257425742574256 episode loss:  257.07602
episode: 4525 episode reward: 37.0 epsilon: 0.6295984981015124 avg reward (last 100): 20.495049504950494 episode loss:  210.96785
episode: 4526 episode reward: 24.0 epsilon: 0.6295355382517022 avg reward (last 100): 20.653465346534652 episode loss:  263.1234
episode: 4527 episode reward: 13.0 epsilon: 0.629472584697877 avg reward (last 100): 20.613861386138613 episode loss:  202.96384
episode: 4528 episode reward: 19.0 epsilon: 0.6294096374394073 avg reward (last 100): 20.663366336633665 episode loss:  237.48468
episode: 4529 episode reward: 13.0 epsilon: 0.6293466964756633 avg reward (last 100): 20.41584

episode: 4586 episode reward: 27.0 epsilon: 0.625769446289177 avg reward (last 100): 20.138613861386137 episode loss:  256.4745
episode: 4587 episode reward: 22.0 epsilon: 0.6257068693445481 avg reward (last 100): 20.198019801980198 episode loss:  258.80722
episode: 4588 episode reward: 10.0 epsilon: 0.6256442986576136 avg reward (last 100): 20.198019801980198 episode loss:  234.1713
episode: 4589 episode reward: 16.0 epsilon: 0.6255817342277479 avg reward (last 100): 20.18811881188119 episode loss:  191.2713
episode: 4590 episode reward: 11.0 epsilon: 0.625519176054325 avg reward (last 100): 20.059405940594058 episode loss:  189.52534
episode: 4591 episode reward: 41.0 epsilon: 0.6254566241367197 avg reward (last 100): 20.346534653465348 episode loss:  210.00377
episode: 4592 episode reward: 12.0 epsilon: 0.625394078474306 avg reward (last 100): 20.06930693069307 episode loss:  164.83653
episode: 4593 episode reward: 32.0 epsilon: 0.6253315390664586 avg reward (last 100): 20.168316831